In [49]:
#Create scenarios for PP location based on "aggregation" and "distance from outfall"
#Import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import math
global str
from pyswmm import Simulation, Nodes, LidControls, Subcatchments, RainGages   

In [50]:
#function to assign the PP in the input files
def assign_clusters_input_file(df,file_name):
#Prepare the text to be added in [LID USAGE]
    names=[]
    selection=[]
    PP_to_assign_last_cluster=df["area_ac"].iloc[-1]
    switch=0
    txt= ""

    for i in PP_ids: #the ID of each cluster in df
        selection=df_subcatch[(df_subcatch['clust100']==i)] #extracts the subcatchments of each cluster
        selection.columns = ["x", "y", "area","clust100"]
        #if it is the last cluster, the area to be assigned is not all the area of the subctachment
        if i == PP_ids[-1]:        
            for row in selection.iterrows(): 
                print(row[1][2])
                print(PP_to_assign_last_cluster)
                #if the subcatchment area is larger than the PP area to assign, then use all the subcatch area
                if row[1][2]> PP_to_assign_last_cluster:

                    if PP_to_assign_last_cluster>0.01: #while there is still PP area to asign
                        txt+=row[0] + " " + 'Porous_Pavements' + " " + str(1)+ " " + str(math.floor(PP_to_assign_last_cluster*43560*100)/100) + " " + str(0)+ " " + str(0)+ " " + str(0)+ " " + str(0) + "\n"
                        PP_to_assign_last_cluster=0
                    break
                #otherwise, assign the PP that is left
                else:            
                    txt+=row[0] + " " + 'Porous_Pavements' + " " + str(1)+ " " + str(math.floor(row[1][2]*43560*100)/100) + " " + str(0)+ " " + str(0)+ " " + str(0)+ " " + str(0) + "\n"
                    PP_to_assign_last_cluster=PP_to_assign_last_cluster-row[1][2]

        #otherwise, assign all the subcatchment area to PP
        else:
            for row in selection.iterrows():  
                txt+=row[0] + " " + 'Porous_Pavements' + " " + str(1)+ " " + str(math.floor(row[1][2]*43560*100)/100) + " " + str(0)+ " " + str(0)+ " " + str(0)+ " " + str(0) + "\n"

    #Identifies the correct location to inject the Text in the input file.
    with open('BSA_Main_Model.inp') as f:
        counter=1
        datafile = f.readlines()
        for line in datafile:
            if '[LID_USAGE]' in line:
                SUBC=counter
            counter=counter+1    

    NEWfile = open(file_name, "w")
    LINEcounter=1       
    for line in datafile:
        NEWfile.write(line)
        if LINEcounter==SUBC+2: #two lines ahead [LID_USAGE], making space for headers
            NEWfile.write(txt)
        LINEcounter=LINEcounter+1 
    

In [51]:
#load required files

#load subcatchments ids for each cluster
subcatch_TEMP= pd.read_csv('cso014_cluster_elements.txt', sep="\t")
subcatch_NAMES=subcatch_TEMP['NAME']
data_subcatch= np.genfromtxt(fname='cso014_cluster_elements.txt')
data_subcatch=np.delete(data_subcatch, 0, axis=0)
data_subcatch=np.delete(data_subcatch, 0, axis=1)
df_subcatch = pd.DataFrame(data_subcatch)
df_subcatch.columns = ["x", "y", "area","clust100"]
df_subcatch.index=subcatch_NAMES
#print(df_subcatch.loc['Parking_E-MH-7650',:])
#print(df_subcatch[df_subcatch['clust100']==10])

#load clusters list and sector
data = np.genfromtxt(fname='cso014_CLUSTERS_S2.txt')
data=np.delete(data, 0, axis=0)
df = pd.DataFrame(data)
df.columns = ["cluster100", "x", "y", "distance_outfall","rank_dist","area", "sector"]

In [53]:
clusters_number=100 #number of clusters
iterations=2 #number of iterations= number of runs
random.seed(1234)
nsect=3 #number of sectors 
arr={}

for i in range(1,nsect+1): #iterating over the sectors
    for j in range(iterations): #number of iterations per sector
        PP_assigned=[] #create an empty list to store the PP area assigned
        PP_ids=[] #create an empty list to store the subcatchments id's to which PP area is assigned
        PP_to_assign=50 #area in acres to be assigned
        df_temp=df[df['sector']==i] #extract a temporal dataframe with clusters in the sector i
        listclusters=list(df_temp.cluster100) #transform to a list
        
        while PP_to_assign>0: #while there is still PP are to asign
            current_clust=listclusters[random.randint(0, len(listclusters)-1)] #pick a cluster randomly
            listclusters.remove(current_clust) #remove the selected cluster from the list (to avoid repetition)
            #if the subcatchment area is larger than the PP area to assign, then use all the subcatch area
            area_cl=float(df_temp[df_temp["cluster100"]==current_clust].area) #
            if area_cl> PP_to_assign:
                PP_assigned.append(PP_to_assign)
                PP_to_assign=0
            #otherwise, assign the PP that is left
            else:            
                PP_to_assign= PP_to_assign-area_cl
                PP_assigned.append(area_cl)
            PP_ids.append(current_clust)
        df3= pd.DataFrame(list(zip(PP_ids, PP_assigned)), columns =['cluster', 'area_ac'])

        file_name='sector_'+str(i)+'_it_'+str(j)+'.inp'
        #Run the function to create a new input file
        assign_clusters_input_file(df3,file_name)

0.184
4.679
0.157
4.495
0.35
4.338
0.423
3.988
1.531
3.565
0.195
2.034
0.615
1.8389999999999997
1.663
1.2239999999999998
0.923
9.212999999999997
0.106
8.289999999999997
0.508
8.183999999999997
0.15
7.6759999999999975
0.392
7.525999999999997
0.143
7.133999999999997
0.33
6.990999999999997
0.725
6.660999999999997
0.7
5.935999999999997
0.855
5.235999999999997
0.303
4.380999999999997
0.364
4.077999999999997
0.557
3.713999999999997
0.243
3.156999999999997
0.154
2.913999999999997
0.771
2.759999999999997
2.64
1.9889999999999972
1.426
4.284000000000001
0.226
2.8580000000000005
2.005
2.6320000000000006
0.13
0.6270000000000007
0.151
0.49700000000000066
0.51
0.34600000000000064
0.972
2.3450000000000006
0.355
1.3730000000000007
0.372
1.0180000000000007
0.131
0.6460000000000007
0.249
0.5150000000000007
0.134
0.2660000000000007
0.201
0.13200000000000067
0.979
1.633868999999998
1.097
0.654868999999998
0.192
2.331000000000003
0.258
2.139000000000003
0.663
1.881000000000003
0.261
1.2180000000000029
0.78